In [3]:
# GPU ON KRKE RUN ALL -> google colab mein kr dena run isko
import re
import random
import nltk
import math
nltk.download('punkt')
import numpy as np
import pandas as pd
# for progress bars
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import joblib
import spacy

In [4]:
# HYPER PARAMETERS

# sequence length for the sequence-target pair
seq_len = 5

# batch size
batch_size = 16

# number of epochs
epochs = 5

# learning rate
lr = 0.001

# clip
clip = 1

# threshold -> for handling unknown words. If the frequency comes out to be less than threshold, then we will replace the token with unk
threshold = 2

In [5]:
# read file
file_name = '../input/dataset/intro-to-nlp-assign3/news-crawl-corpus/train.news'
f = open(file_name)
text = f.read()
f.close()

In [6]:
!python3 -m spacy download fr_core_news_sm
spacy_fr = spacy.load('fr_core_news_sm')

In [7]:
text = text.lower()
lst = nltk.tokenize.sent_tokenize(text)

sentences = []
for sent in lst:
    sentences.append([tok.text for tok in spacy_fr.tokenizer(sent)])

In [11]:
# frequency count
freq = {}
for sent in sentences:
    for word in sent:
        if freq.get(word) == None:
            freq[word] = 1
        else:
            freq[word] += 1

pure_sentences = []
for sent in sentences:
    pure = ''
    for word in sent:
        if freq[word] >= threshold:
            pure += word
        else:
            pure += 'unk'

        pure += ' '
    pure_sentences.append(pure)

In [14]:
# creating sequence-target pair and also storing all the distinct words
seq = []
dist_words = set()
for string in pure_sentences:
    # we will take seq_len+1 and then divide this into input_sequence and output_sequence     
    sent = string.split()
    if len(sent) >= seq_len:
        for i in range(seq_len,len(sent)):
            seq.append(" ".join(sent[i-seq_len:i+1]))
        
    for word in sent:
        dist_words.add(word)
        
# vocabulary size
vocab_size = len(dist_words)
    
inp = []
out = []
for sq in seq:
    inp.append(" ".join(sq.split()[:-1]))  # from first word to last second word 
    out.append(" ".join(sq.split()[1:]))   # from second word to last word

    
# creating word_to_index and index_to_word dictionary
word_to_index = {}
index_to_word = {}
for cnt,word in enumerate(dist_words):
    word_to_index[word] = cnt
    index_to_word[cnt] = word

In [15]:
joblib.dump(word_to_index,'word_to_index_q1.pkl')
joblib.dump(index_to_word,'index_to_word_q1.pkl')

In [16]:
print(vocab_size, len(inp))
print(seq[0]+'\n'+inp[0]+'\n'+out[0])

In [17]:
# As we have got the index for each word, now convert inp and out to their corresponding word's indices
inp_index = []
out_index = []
for i in range(len(inp)):
    inp_index.append([word_to_index[word] for word in inp[i].split()])
    out_index.append([word_to_index[word] for word in out[i].split()])
    
inp_index = np.array(inp_index)
out_index = np.array(out_index)

In [18]:
# creating batches
def make_batch(inp, out, batch_size):
    ind = 0
    for n in range(batch_size, len(inp), batch_size):
        x = inp[ind:n,:]
        y = out[ind:n,:]
        ind = n
        # yield is used to return from a function without destroying the states of its local variable 
        # and when the function is called, the execution starts from the last yield statement.         
        yield x, y

In [19]:
# Declaring out Model
class WordLSTM(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=4, drop_prob=0.3, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding(vocab_size, 200)

        ## define the LSTM
        self.lstm = nn.LSTM(200, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''

        ## pass input through embedding layer
        embedded = self.emb_layer(x)     
        
        ## Get the outputs and the new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(lstm_output)
        
        #out = out.contiguous().view(-1, self.n_hidden) 
        out = out.reshape(-1, self.n_hidden) 

        ## put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available
        if (torch.cuda.is_available()):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        # if GPU is not available
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [20]:
# instantiate the model
net = WordLSTM()
# push the model to GPU (avoid it if you are not using the GPU)
net.cuda()
print(net)

In [ ]:
def train(net, epochs, batch_size, lr, clip):
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    net.cuda()
    
    net.train()
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        for x, y in make_batch(inp_index, out_index, batch_size):
            inputs, targets = torch.from_numpy(x).cuda(), torch.from_numpy(y).cuda()
            # detach hidden states
            h = tuple([each.data for each in h])
            net.zero_grad()
            output, h = net(inputs, h)
            loss = criterion(output, targets.view(-1))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step() 
        
        print('Epoch {} done!'.format(e+1))

In [ ]:
# train the model & save it
train(net, epochs, batch_size, lr, clip)
torch.save(net.state_dict(),'q1_french.pt')

In [ ]:
def perplexity(sentence):
    # push to GPU
    net.cuda()
    net.eval()

    # batch size is 1
    h = net.init_hidden(1)
    
    sentence = sentence.lower()
    sentence = re.sub("[^a-z']", " ", sentence)
    indices = []
    for token in sentence.split():
        if word_to_index.get(token) == None:
            indices.append(word_to_index['unk'])
        else:
            indices.append(word_to_index[token])

    length = len(indices)
    lst = np.array([indices])

    # tensor inputs
    inputs = torch.from_numpy(lst).cuda()

    # get the output of the model
    out, h = net(inputs, h)

    # get the token probabilities
    p = F.softmax(out, dim=1).data
    p = p.cpu().numpy()

    logarithm_sum = 0.0
    for i,ind in enumerate(indices):
        logarithm_sum += (math.log(p[i][ind]))/length
        
    # we already took division by length, so no need to divide here by length     
    return math.exp(-logarithm_sum)
    

In [ ]:
p = perplexity('the results are acceptable.')
print(p)